<p style="border:2px solid black"> </p>
<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.8cm"></p>
<center>
<font size="6"><b>Understanding Music Listening Habits</b></font>
<p style="margin-bottom:-0.1cm"></p>
<font size="6"><b>Using Large-scale Smartphone Data</b>  </font>

<p style="margin-bottom:0.5cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<p style="margin-bottom:1cm"></p>
<font size="5"><b>Preliminary Data Analysis</b></font>
<br>
<font size="3"><b></b></font>
</center>
<p style="margin-bottom:0.4cm"></p>
<p style="border:2px solid black"> </p>

# Initialization
<hr style="border:2px solid black"></hr>


The initializaiton procedure is defined in the notebook [Initialization](initialization.ipynb).

In [1]:
%run initialization.ipynb

# Create a Spark Session
<hr style="border:2px solid black"></hr>


In [2]:
#%% Initialize a Spark session.
spark = t.spark.create_session('Music_Activity')

# Set this for faster conversion from Spark to pandas.
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Creating a Spark session.
	Execution time: 6.48568 s.


# Load Data
<hr style="border:2px solid black"></hr>


Get the paths of data files and folders:

In [4]:
data_root, data_subfolders, data_files = t.get_paths_raw_data()

Getting the data paths.
	Number of folders: 62
	Number of files: 275621


Load the raw and full data. The *raw* data contains the data in an unaltered table. The the *full* data contains the data, where the the columns `devices` and `tracks` are exploded, så that each row corresponds to a playback of one song on one device.

In [5]:
df_raw = t.load_data_from_files(data_root, spark, method='avro')
df_full = t.format_dataframe(df_raw)

Loading data from path: "/data/work/src/musicactivity".
	Execution time: 8.43431 s.
Formatting dataframe:
	Exploding columns containing lists.
	Flattening the dataframe schema.
	Execution time: 0.24210


Now, we load a sample dataframe containing 1e6 randomly selected observations to use to gain a faster insight into the data.

In [3]:
path_df_1E6 = Config.Path.music_data_raw_sample_1E6_root

df_1E6 = t.load_data_from_files(path_df_1E6, spark)

Loading data from path: "/data/work/shared/s001284/Music_Project/resources/data/df_sample_1E6.parquet".
	Execution time: 5.74674 s.


Cache the dataframes (use only if needed):

In [7]:
# df_1E6.cache()
# number_of_rows_df_1E6 = t.spark.count_rows(df_1E6)

# Preliminary data analysis
<hr style="border:2px solid black"></hr>

Take the first 100 rows of the small database and transform it to Pandas for the comfort of viewing.

In [8]:
df_sample_pd = df_1E6.limit(100).toPandas()

Get the description of the the columns from the writer schema in the avro files:

In [12]:
column_docs = t.get_avro_docs(data_files[0])
for column in column_docs:
    if len(column[1]) > 0:
        w.printmd(f'**{column[0]}**: *{column[1]}*')
    else:
        w.printmd(f'**{column[0]}**: *-*')

**id**: *Activity UUID*

**deleted_time**: *If this activity has been deleted, this is the timestamp when it occurred*

**useruuid**: *User UUID*

**start_time**: *Start time*

**end_time**: *End time*

...

**tracks_album**: *-*

**tracks_title**: *-*

**tracks_uri**: *-*

**tracks_player**: *-*

**tracks_id**: *-*

Show the top rows:

In [13]:
df_sample_pd.head(30)

,id,deleted_time,useruuid,start_time,end_time,devices_name,devices_type,devices_id,tracks_start_time,tracks_end_time,tracks_artist,tracks_album,tracks_title,tracks_uri,tracks_player,tracks_id,yearmonth
0,96c4958e-781b-477c-9823-6a6fc5cdfed0-2016-10-16,,217abb4b-cf26-4b28-a1af-bfa0565f64d8,2016-10-16T11:14:24.281+02:00,2016-10-16T11:58:41.070+02:00,D6503,PHONE,30575b739121b7a7,2016-10-16T11:42:33.218+02:00,2016-10-16T11:45:42.689+02:00,<unknown>,mp3,Maroon 5 - Maps (Lyric Video),content://media/external/audio/media/87943,Walkman,,201610
1,a8eec483-caa5-4297-ab53-12dfae409b44-2016-10-16,,d98eb3db-d1b5-47a3-a6cd-7f8b651913e0,2016-10-16T12:05:13.901+02:00,2016-10-16T12:15:11.685+02:00,D5503,PHONE,c9d2ad0a5e3b17a,2016-10-16T12:08:19.172+02:00,2016-10-16T12:09:07.304+02:00,ROMAIN VIRGO,CORNER SHOP RIDDIM [FULL PROMO] - 21ST HAPILOS...,BEAT YOU DOWN,content://media/external/audio/media/18753,Walkman,,201610
2,741a6351-cd04-47ce-bfa0-c906848ca52f-2016-10-16,,07b500bf-b975-4fc1-b203-d25973809a8a,2016-10-16T12:07:44.666+05:00,2016-10-16T14:45:58.478+05:00,E6633,PHONE,84b6ce0d67c6ab0c,2016-10-16T13:10:43.036+05:00,2016-10-16T13:14:48.150+05:00,Ma muzik,Music,Claudio Ismael - A tua escolha 2013,content://media/external/audio/media/2549,Walkman,,201610
3,2720ff80-e5f3-4a92-a8cd-5f7840b2743a-2016-10-16,,0f0742d4-454f-4234-a7aa-6980eeedf5fa,2016-10-16T15:37:42.058+09:00,2016-10-16T19:30:59.929+09:00,SO-03F,PHONE,5e0957ef6094f22f,2016-10-16T16:38:21.892+09:00,2016-10-16T16:44:09.193+09:00,<unknown>,audio,DaizyStripper ー Brilliant Days．,content://media/external/audio/media/4816,Walkman,,201610
4,2868bc0a-1fb3-4020-8706-811ae31f9f92-2016-10-16,,5f500365-af05-4ff0-ae64-abacd71eac06,2016-10-16T10:59:54.106+02:00,2016-10-16T11:21:26.814+02:00,E6653,PHONE,a4b2701122758f84,2016-10-16T11:03:12.722+02:00,2016-10-16T11:08:09.438+02:00,<unknown>,Music,Kizomba - Philipe Monteiro - Alta Segurança,content://media/external/audio/media/220,Walkman,,201610
5,65290fff-d70c-41c7-9ca9-f2b22cfa5fdc-2016-10-16,,62a8ea8f-9a27-4116-b880-a4050bad5926,2016-10-15T23:50:06.296-04:00,2016-10-16T01:14:11.333-04:00,D6616,PHONE,1b4054332c02838f,2016-10-16T00:21:04.592-04:00,2016-10-16T00:25:13.417-04:00,Cali y El Dandee Ft. Juan Magan y Sebastian Yatra,www.planetaexitos.com,Por Fin Te Encontré,content://media/external/audio/media/73063,Walkman,,201610
6,2abfb220-1b29-4df0-a395-9490947ccae6-2016-10-16,2016-10-16T11:55:22.114Z,35389166-37c2-4cd8-8ff1-152deeb0c11a,2016-10-16T17:26:08.565+09:00,2016-10-16T19:28:11.968+09:00,SO-02G,PHONE,b53e0801692fedf8,2016-10-16T17:42:59.463+09:00,2016-10-16T17:46:37.396+09:00,Sum 41,"The Best of Sum 41: 8 Years of Blood, Sake and...",Handle This,content://media/external/audio/media/8402,Walkman,,201610
7,694dfcc0-5406-44ca-8739-c0c57b365dd5-2016-10-16,,cd35971e-fdef-4b3e-909e-7c10a92e9718,2016-10-16T13:22:24.687+02:00,2016-10-16T13:39:22.476+02:00,D6603,PHONE,39d083c10ff8583f,2016-10-16T13:38:49.599+02:00,2016-10-16T13:39:22.476+02:00,A$AP Rocky,AT.LONG.LAST.A$AP,Lord Pretty Flacko Jodye 2 (LPFJ2),,Spotify,spotify:track:1j6kDJttn6wbVyMaM42Nxm,201610
8,9dda8f1a-4409-40ef-b569-e7dcfdd45a89-2016-10-16,,9c73877f-0277-4a73-bc25-f7910dd3bc7e,2016-10-16T02:52:18.382+02:00,2016-10-16T03:38:40.497+02:00,D6503,PHONE,29ff78f4b611dee3,2016-10-16T03:32:16.784+02:00,2016-10-16T03:36:07.264+02:00,Panet.co.il_Rabih-Baroud,www.panet.co.il,Taibo,content://media/external/audio/media/338585,Walkman,,201610
9,1c06a1a4-11e5-479d-b4eb-d430e56e8b41-2016-10-16,,77d2f9e5-68f0-4182-9ca9-dd8fdf128989,2016-10-16T17:06:48.356+05:00,2016-10-16T17:07:24.935+05:00,E6553,PHONE,8bbf450ec7eceee9,2016-10-16T17:06:48.356+05:00,2016-10-16T17:07:24.935+05:00,<unknown>,Download,TairoJ'étais prêt,content://media/external/audio/media/27003,Walkman,,201610


## Number of rows

Raw Database:

In [14]:
number_of_rows_raw = df_raw.count()
print(f'Number of rows in the raw database: {number_of_rows_raw}')

Number of rows in the raw database: 627101296


Full Database:

In [15]:
number_of_rows_full = df_full.count()
print(f'Number of rows in the full database: {number_of_rows_full}')

Number of rows in the full database: 3571278161


In [14]:
# For execution of the notebook without counting
number_of_rows_raw = 627101296
number_of_rows_full = 3571278161

## Activity ID

Detecting nonempty cells:

In [17]:
start_time = time.time()

n_empty_id_cells = (
    df_raw
    .where(
        (f.col('id').isNull())
        | (f.col('id') == '')
        | (f.col('id') == '<unknown>')
    )
    .count()
)

print(f'Number of empty "id" cells: {n_empty_id_cells}.')
print(f'Execution time: {time.time() - start_time:.5f} s.')

Number of empty id cells: 0.
Execution time: 338.16046 s.


Detecting duplicates:

In [19]:
start_time = time.time()

(df_raw
 .groupBy(f.col('id'))
 .count()
 .orderBy(f.desc('count'))                    
 .show(5)
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

+--------------------+-----+
|                  id|count|
+--------------------+-----+
|8c71293b-fe42-432...|  410|
|ff29d186-0614-42d...|  386|
|934c3a82-a73c-4ec...|  314|
|a2ca4895-8074-426...|  301|
|ae2fedfb-ef75-4ec...|  283|
+--------------------+-----+
only showing top 5 rows

Execution time: 585.76851 s.


Number of duplicate activity ID's:

In [15]:
start_time = time.time()

number_of_duplicate_activity_ids = (
    df_raw
    .groupBy(f.col('id'))
    .count()
    .where(f.col('count') > 1)
    .count()
)

print(f"Number of duplicated activity ID's: {number_of_duplicate_activity_ids}.")
print(f'That constitutes '
      f'{number_of_duplicate_activity_ids / number_of_rows_raw * 100:.5f} % '
      f'of the total number of cells.')
print(f'Execution time: {time.time() - start_time:.5f} s.')

Number of duplicated activity ID's: 46360934.
That constitutes 7.39289 % of the total number of cells.
Execution time: 580.64515 s.


## Activity Deleted

In [16]:
start_time = time.time()

n_nonempty_deleted_time_cells = (
    df_raw
    .where(
        (f.col('deleted_time').isNotNull())
        & (f.col('deleted_time') != '')
    )
    .count()
)

print(f'Number of non-empty "deleted_time" cells: {n_nonempty_deleted_time_cells}.')
print(f'That constitutes '
      f'{n_nonempty_deleted_time_cells / number_of_rows_raw * 100:.5f} % '
      f'of the total number of cells.')
print(f'Execution time: {time.time() - start_time:.5f} s.')

Number of non-empty "deleted_time" cells: 52764881.
That constitutes 8.41409 % of the total number of cells.
Execution time: 331.54952 s.


Around 8.5 % of all the activities have the "Deleted time" property set. What to do with them:
 * Nothing
 * Delete the column
 * Delete the rows where the property is set

## Device types

In [17]:
start_time = time.time()

(df_full
 .groupBy(f.col('devices_type'))
 .count()
 .orderBy(f.desc('count'))                    
 .show()
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

+------------+----------+
|devices_type|     count|
+------------+----------+
|       PHONE|3561473696|
|        null|   7633807|
|     UNKNOWN|   2170658|
+------------+----------+

Execution time: 953.80290 s.


Almost all the devices are phones. It could be reasonable remove the column.

## Track players

In [18]:
start_time = time.time()

(df_full
 .groupBy(f.col('tracks_player'))
 .count()
 .orderBy(f.desc('count'))                    
 .show()
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

+-------------+----------+
|tracks_player|     count|
+-------------+----------+
|      Walkman|2428561686|
|      Spotify| 710807090|
|             | 431909382|
|        Music|         3|
+-------------+----------+

Execution time: 997.08188 s.


## Track ID vs Track URI

In [21]:
start_time = time.time()

n_track_id_and_uri_nonempty = (
    df_full
    .where(
        (f.col('tracks_uri') != '')
        & (f.col('tracks_uri').isNotNull())
        & (f.col('tracks_id') != '')     
        & (f.col('tracks_id').isNotNull())
    )
    .count()
)

print(f'Number of rows where both "tracks_id" and '
      f'"tracks_uri" are full: {n_track_id_and_uri_nonempty}.')
print(f'Execution time: {time.time() - start_time:.5f} s.')

Number of rows where both "tracks_id" and "tracks_uri" are full: 0.
Execution time: 925.96000 s.


The Track ID and Track URI are never full at the same time. Therefore, the columns could be merged into one column.

Also, the information might be redundant. 

We could also split the dataset into a couple of databases. Does it make sense? Maybe not.
 

## Timestamps

In [59]:
from pyspark.sql.types import LongType


df_test = df_1E6.limit(100)
df2 = (
    df_test
    .withColumn('start_time',
                f.to_timestamp('start_time',
                                 "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"))
    .withColumn('end_time',
                f.to_timestamp('end_time',
                                 "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"))
    .withColumn('tracks_start_time',
                f.to_timestamp('tracks_start_time',
                                 "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"))
    .withColumn('tracks_end_time',
                f.to_timestamp('tracks_end_time',
                                 "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"))
    .withColumn('activity_duration_seconds',
                f.col('end_time').cast(LongType()) - f.col('start_time').cast(LongType()))
)

df2.toPandas().head()

[('id', 'string'), ('deleted_time', 'string'), ('useruuid', 'string'), ('start_time', 'timestamp'), ('end_time', 'timestamp'), ('devices_name', 'string'), ('devices_type', 'string'), ('devices_id', 'string'), ('tracks_start_time', 'timestamp'), ('tracks_end_time', 'timestamp'), ('tracks_artist', 'string'), ('tracks_album', 'string'), ('tracks_title', 'string'), ('tracks_uri', 'string'), ('tracks_player', 'string'), ('tracks_id', 'string'), ('yearmonth', 'int'), ('activity_duration_seconds', 'bigint')]


,id,deleted_time,useruuid,start_time,end_time,devices_name,devices_type,devices_id,tracks_start_time,tracks_end_time,tracks_artist,tracks_album,tracks_title,tracks_uri,tracks_player,tracks_id,yearmonth,activity_duration_seconds
0,96c4958e-781b-477c-9823-6a6fc5cdfed0-2016-10-16,,217abb4b-cf26-4b28-a1af-bfa0565f64d8,2016-10-16 11:14:24.281,2016-10-16 11:58:41.070,D6503,PHONE,30575b739121b7a7,2016-10-16 11:42:33.218,2016-10-16 11:45:42.689,<unknown>,mp3,Maroon 5 - Maps (Lyric Video),content://media/external/audio/media/87943,Walkman,,201610,2657
1,a8eec483-caa5-4297-ab53-12dfae409b44-2016-10-16,,d98eb3db-d1b5-47a3-a6cd-7f8b651913e0,2016-10-16 12:05:13.901,2016-10-16 12:15:11.685,D5503,PHONE,c9d2ad0a5e3b17a,2016-10-16 12:08:19.172,2016-10-16 12:09:07.304,ROMAIN VIRGO,CORNER SHOP RIDDIM [FULL PROMO] - 21ST HAPILOS...,BEAT YOU DOWN,content://media/external/audio/media/18753,Walkman,,201610,598
2,741a6351-cd04-47ce-bfa0-c906848ca52f-2016-10-16,,07b500bf-b975-4fc1-b203-d25973809a8a,2016-10-16 09:07:44.666,2016-10-16 11:45:58.478,E6633,PHONE,84b6ce0d67c6ab0c,2016-10-16 10:10:43.036,2016-10-16 10:14:48.150,Ma muzik,Music,Claudio Ismael - A tua escolha 2013,content://media/external/audio/media/2549,Walkman,,201610,9494
3,2720ff80-e5f3-4a92-a8cd-5f7840b2743a-2016-10-16,,0f0742d4-454f-4234-a7aa-6980eeedf5fa,2016-10-16 08:37:42.058,2016-10-16 12:30:59.929,SO-03F,PHONE,5e0957ef6094f22f,2016-10-16 09:38:21.892,2016-10-16 09:44:09.193,<unknown>,audio,DaizyStripper ー Brilliant Days．,content://media/external/audio/media/4816,Walkman,,201610,13997
4,2868bc0a-1fb3-4020-8706-811ae31f9f92-2016-10-16,,5f500365-af05-4ff0-ae64-abacd71eac06,2016-10-16 10:59:54.106,2016-10-16 11:21:26.814,E6653,PHONE,a4b2701122758f84,2016-10-16 11:03:12.722,2016-10-16 11:08:09.438,<unknown>,Music,Kizomba - Philipe Monteiro - Alta Segurança,content://media/external/audio/media/220,Walkman,,201610,1292


## Merging Track ID and Track URI

In [43]:
df2 = (
    df2
    .withColumn('track_id',
                f.concat(f.col('tracks_id'), 
                         f.col('tracks_uri')))
)

df2.toPandas().head(20)

,id,deleted_time,useruuid,start_time,end_time,devices_name,devices_type,devices_id,tracks_start_time,tracks_end_time,tracks_artist,tracks_album,tracks_title,tracks_uri,tracks_player,tracks_id,yearmonth,activity_duration_seconds,track_id
0,96c4958e-781b-477c-9823-6a6fc5cdfed0-2016-10-16,,217abb4b-cf26-4b28-a1af-bfa0565f64d8,1476609264,1476611921,D6503,PHONE,30575b739121b7a7,1476610953,1476611142,<unknown>,mp3,Maroon 5 - Maps (Lyric Video),content://media/external/audio/media/87943,Walkman,,201610,2657,content://media/external/audio/media/87943
1,a8eec483-caa5-4297-ab53-12dfae409b44-2016-10-16,,d98eb3db-d1b5-47a3-a6cd-7f8b651913e0,1476612313,1476612911,D5503,PHONE,c9d2ad0a5e3b17a,1476612499,1476612547,ROMAIN VIRGO,CORNER SHOP RIDDIM [FULL PROMO] - 21ST HAPILOS...,BEAT YOU DOWN,content://media/external/audio/media/18753,Walkman,,201610,598,content://media/external/audio/media/18753
2,741a6351-cd04-47ce-bfa0-c906848ca52f-2016-10-16,,07b500bf-b975-4fc1-b203-d25973809a8a,1476601664,1476611158,E6633,PHONE,84b6ce0d67c6ab0c,1476605443,1476605688,Ma muzik,Music,Claudio Ismael - A tua escolha 2013,content://media/external/audio/media/2549,Walkman,,201610,9494,content://media/external/audio/media/2549
3,2720ff80-e5f3-4a92-a8cd-5f7840b2743a-2016-10-16,,0f0742d4-454f-4234-a7aa-6980eeedf5fa,1476599862,1476613859,SO-03F,PHONE,5e0957ef6094f22f,1476603501,1476603849,<unknown>,audio,DaizyStripper ー Brilliant Days．,content://media/external/audio/media/4816,Walkman,,201610,13997,content://media/external/audio/media/4816
4,2868bc0a-1fb3-4020-8706-811ae31f9f92-2016-10-16,,5f500365-af05-4ff0-ae64-abacd71eac06,1476608394,1476609686,E6653,PHONE,a4b2701122758f84,1476608592,1476608889,<unknown>,Music,Kizomba - Philipe Monteiro - Alta Segurança,content://media/external/audio/media/220,Walkman,,201610,1292,content://media/external/audio/media/220
5,65290fff-d70c-41c7-9ca9-f2b22cfa5fdc-2016-10-16,,62a8ea8f-9a27-4116-b880-a4050bad5926,1476589806,1476594851,D6616,PHONE,1b4054332c02838f,1476591664,1476591913,Cali y El Dandee Ft. Juan Magan y Sebastian Yatra,www.planetaexitos.com,Por Fin Te Encontré,content://media/external/audio/media/73063,Walkman,,201610,5045,content://media/external/audio/media/73063
6,2abfb220-1b29-4df0-a395-9490947ccae6-2016-10-16,2016-10-16T11:55:22.114Z,35389166-37c2-4cd8-8ff1-152deeb0c11a,1476606368,1476613691,SO-02G,PHONE,b53e0801692fedf8,1476607379,1476607597,Sum 41,"The Best of Sum 41: 8 Years of Blood, Sake and...",Handle This,content://media/external/audio/media/8402,Walkman,,201610,7323,content://media/external/audio/media/8402
7,694dfcc0-5406-44ca-8739-c0c57b365dd5-2016-10-16,,cd35971e-fdef-4b3e-909e-7c10a92e9718,1476616944,1476617962,D6603,PHONE,39d083c10ff8583f,1476617929,1476617962,A$AP Rocky,AT.LONG.LAST.A$AP,Lord Pretty Flacko Jodye 2 (LPFJ2),,Spotify,spotify:track:1j6kDJttn6wbVyMaM42Nxm,201610,1018,spotify:track:1j6kDJttn6wbVyMaM42Nxm
8,9dda8f1a-4409-40ef-b569-e7dcfdd45a89-2016-10-16,,9c73877f-0277-4a73-bc25-f7910dd3bc7e,1476579138,1476581920,D6503,PHONE,29ff78f4b611dee3,1476581536,1476581767,Panet.co.il_Rabih-Baroud,www.panet.co.il,Taibo,content://media/external/audio/media/338585,Walkman,,201610,2782,content://media/external/audio/media/338585
9,1c06a1a4-11e5-479d-b4eb-d430e56e8b41-2016-10-16,,77d2f9e5-68f0-4182-9ca9-dd8fdf128989,1476619608,1476619644,E6553,PHONE,8bbf450ec7eceee9,1476619608,1476619644,<unknown>,Download,TairoJ'étais prêt,content://media/external/audio/media/27003,Walkman,,201610,36,content://media/external/audio/media/27003


# Cleaning
<hr style="border:2px solid black"></hr>

## Replacing start and end time with start time and duration

In [45]:
time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

df_clean = (
    df_sample_raw
    .withColumn('start_time',
                f.to_timestamp('start_time', time_format))
    .withColumn('end_time',
                f.to_timestamp('end_time', time_format))
    .withColumn('activity_duration',
                f.col('end_time').cast(LongType()) 
                - f.col('start_time').cast(LongType()))   
    .withColumn('tracks_start_time',
                f.to_timestamp('tracks_start_time', time_format))
    .withColumn('tracks_end_time',
                f.to_timestamp('tracks_end_time', time_format))
    .withColumn('track_duration',
                f.col('tracks_end_time').cast(LongType()) 
                - f.col('tracks_start_time').cast(LongType()))
    .drop('end_time')
    .drop('tracks_end_time')
)

df_clean_activity_id.limit(10).toPandas().head(10)

,id,deleted_time,useruuid,start_time,devices_name,devices_type,devices_id,tracks_start_time,tracks_end_time,tracks_artist,tracks_album,tracks_title,tracks_uri,tracks_player,tracks_id,yearmonth,activity_duration
0,96c4958e-781b-477c-9823-6a6fc5cdfed0-2016-10-16,,217abb4b-cf26-4b28-a1af-bfa0565f64d8,2016-10-16 11:14:24.281,D6503,PHONE,30575b739121b7a7,2016-10-16T11:42:33.218+02:00,2016-10-16T11:45:42.689+02:00,<unknown>,mp3,Maroon 5 - Maps (Lyric Video),content://media/external/audio/media/87943,Walkman,,201610,2657
1,a8eec483-caa5-4297-ab53-12dfae409b44-2016-10-16,,d98eb3db-d1b5-47a3-a6cd-7f8b651913e0,2016-10-16 12:05:13.901,D5503,PHONE,c9d2ad0a5e3b17a,2016-10-16T12:08:19.172+02:00,2016-10-16T12:09:07.304+02:00,ROMAIN VIRGO,CORNER SHOP RIDDIM [FULL PROMO] - 21ST HAPILOS...,BEAT YOU DOWN,content://media/external/audio/media/18753,Walkman,,201610,598
2,741a6351-cd04-47ce-bfa0-c906848ca52f-2016-10-16,,07b500bf-b975-4fc1-b203-d25973809a8a,2016-10-16 09:07:44.666,E6633,PHONE,84b6ce0d67c6ab0c,2016-10-16T13:10:43.036+05:00,2016-10-16T13:14:48.150+05:00,Ma muzik,Music,Claudio Ismael - A tua escolha 2013,content://media/external/audio/media/2549,Walkman,,201610,9494


## Dropping duplicates of activity id (keeping the most recent)

In [13]:
number_of_records = 3.5e9

df_sample_raw = df_raw.sample(fracti,uhion=1e5 / number_of_records,
                              withReplacement=False)


In [15]:
n_rows_before = df_sample_raw.count()

KeyboardInterrupt: 

In [ ]:
print(f'Rows before: {n_rows_before}')

In [17]:
from pyspark.sql import Window

start_time = time.time()

time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

# Define a partitioning by activity id
window_id = Window.partitionBy('id')

# Drop
df_clean = (
    df_sample_raw
    
    # Convert the start time to timestamp.
    .withColumn('start_time',
                f.to_timestamp('start_time', time_format)) 
    
    # Save the latest start time for each activity ID in a 
    # separate column.
    .withColumn('latest_start_time', f.max('start_time').over(window_id))
    
    # Delete the column containing the lates start time.
    .drop('latest_start_time')
    
    # Get only the first occurrence of each activity ID.
    .dropDuplicates('id')
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

df_clean.limit(5).toPandas().head()

Py4JError: An error occurred while calling z:org.apache.spark.api.python.PythonUtils.toSeq. Trace:
py4j.Py4JException: Method toSeq([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:745)



In [ ]:
n_rows_after = df_clean.count()

print(f'Rows before: {n_rows_before}')
print(f'Rows after: {n_rows_after}')

In [12]:
w = Window.partitionBy('id')
w_t = Window.partitionBy('start_time')
b = (
    df_sample_raw
    .withColumn('id_occurrences', f.count('id').over(w))
    .where(f.col('id_occurrences') > 1)
    .groupBy('id')
    .agg(f.countDistinct('start_time').alias('n_start_time'),
         f.min('start_time').alias('min_start_time'),
         f.max('start_time').alias('max_start_time'))
    .sort(f.desc('n_start_time'))
#     .select('id', 'id_occurrences', 'start_time', 'time_occurrences')
)


b.limit(5).toPandas().head(5)

,id,n_start_time,min_start_time,max_start_time
0,02583b73-a5d6-4fa4-84d2-a78d5bc966e1-2016-09-01,1,2016-09-01T23:07:38.197+02:00,2016-09-01T23:07:38.197+02:00
1,858e79d3-86cf-45f9-aa86-4e0dd3601899-2016-03-30,1,2016-03-31T01:40:02.784+09:00,2016-03-31T01:40:02.784+09:00
2,9b364b08-2188-4e03-9841-580335c50617-2016-12-13,1,2016-12-13T08:12:55.715Z,2016-12-13T08:12:55.715Z
3,c1c38e4b-b2d9-4251-bb34-bf1969272155-2015-07-29,1,2015-07-29T09:14:27.522+09:00,2015-07-29T09:14:27.522+09:00
4,0f0b55d7-0e31-4c95-92c9-d5d6cb910524-2015-06-03,1,2015-06-03T18:25:55.452+08:00,2015-06-03T18:25:55.452+08:00


In [11]:
df_clean_activity_id = (
    df_clean_activity_id
    .groupBy('id')
    .withCol

)

df_clean_activity_id = (
    df_clean_activity_id
    .orderBy(f.desc('start_time'))
#     .withColumn('start_time',
#                 f.to_timestamp('start_time', time_format))
#     .withColumn('end_time',
#                 f.to_timestamp('end_time', time_format))
#     .withColumn('activity_duration',
#                 f.col('end_time').cast(LongType()) 
#                 - f.col('start_time').cast(LongType()))
    
#     .withColumn('tracks_start_time',
#                 f.to_timestamp('tracks_start_time', time_format))
#     .withColumn('tracks_end_time',
#                 f.to_timestamp('tracks_end_time', time_format))
#     .withColumn('track_duration',
#                 f.col('tracks_end_time').cast(LongType()) 
#                 - f.col('tracks_start_time').cast(LongType()))
#     .drop('end_time')
#     .drop('tracks_end_time')
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

df_clean_activity_id.limit(3).toPandas().head()

KeyboardInterrupt: 

## Drop redundant columns

Let's remove the columns:

* `deleted_time`
* `devices_type`
* `yearmonth`

In [7]:
df_clean = (
    df_full
    .drop('deleted_time')
    .drop('devices_type')
    .drop('yearmonth')
)

df_clean.limit(3).toPandas().head()

,id,useruuid,start_time,end_time,devices_name,devices_id,tracks_start_time,tracks_end_time,tracks_artist,tracks_album,tracks_title,tracks_uri,tracks_player,tracks_id
0,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16T18:31:56.512+08:00,2016-10-16T18:52:01.608+08:00,E6853,e6211e25cf8fd7bc,2016-10-16T18:31:56.512+08:00,2016-10-16T18:35:03.980+08:00,<unknown>,bluetooth,GINTAMA 曇天,content://media/external/audio/media/1600,Walkman,
1,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16T18:31:56.512+08:00,2016-10-16T18:52:01.608+08:00,E6853,e6211e25cf8fd7bc,2016-10-16T18:35:03.980+08:00,2016-10-16T18:36:42.627+08:00,<unknown>,bluetooth,GunXSword,content://media/external/audio/media/1601,Walkman,
2,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16T18:31:56.512+08:00,2016-10-16T18:52:01.608+08:00,E6853,e6211e25cf8fd7bc,2016-10-16T18:36:42.627+08:00,2016-10-16T18:37:17.290+08:00,<unknown>,bluetooth,i jubeat knit~LOVE&JOY-木村由姬,content://media/external/audio/media/1603,Walkman,


## Merging Track ID and Track URI

In [9]:
df_clean = (
    df_clean
    .withColumn('track_id',
                f.concat(f.col('tracks_id'), 
                         f.col('tracks_uri')))
    .drop('tracks_id')
    .drop('tracks_uri')
)

df_clean.limit(3).toPandas().head()

,id,useruuid,start_time,devices_name,devices_id,tracks_start_time,tracks_artist,tracks_album,tracks_title,tracks_player,activity_duration,track_duration,track_id
0,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16 12:31:56.512,E6853,e6211e25cf8fd7bc,2016-10-16 12:31:56.512,<unknown>,bluetooth,GINTAMA 曇天,Walkman,1205,187,content://media/external/audio/media/1600
1,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16 12:31:56.512,E6853,e6211e25cf8fd7bc,2016-10-16 12:35:03.980,<unknown>,bluetooth,GunXSword,Walkman,1205,99,content://media/external/audio/media/1601
2,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16 12:31:56.512,E6853,e6211e25cf8fd7bc,2016-10-16 12:36:42.627,<unknown>,bluetooth,i jubeat knit~LOVE&JOY-木村由姬,Walkman,1205,35,content://media/external/audio/media/1603


## Rename and organize columns

In [10]:
start_time = time.time()

df_clean = (
    df_clean
    .withColumnRenamed('id', 'activity_id')
    .withColumnRenamed('useruuid', 'user_id')
    .withColumnRenamed('start_time', 'activity_start_time')
    .withColumnRenamed('devices_name', 'device_name')
    .withColumnRenamed('devices_id', 'device_id')
    .withColumnRenamed('tracks_start_time', 'track_start_time')
    .withColumnRenamed('tracks_artist', 'track_artist')
    .withColumnRenamed('tracks_album', 'track_album')
    .withColumnRenamed('tracks_title', 'track_title')
    .withColumnRenamed('tracks_player', 'track_player')
    .select('activity_id', 'activity_start_time', 'activity_duration',
            'user_id',
            'device_id', 'device_name',
            'track_id', 'track_artist', 'track_title', 'track_album',
            'track_player', 'track_start_time', 'track_duration')
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

df_clean.limit(3).toPandas().head()

Execution time: 0.08106 s.


,activity_id,activity_start_time,activity_duration,user_id,device_id,device_name,track_id,track_artist,track_title,track_album,track_player,track_start_time,track_duration
0,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,2016-10-16 12:31:56.512,1205,92e17ec9-982c-4e02-8ce4-76b19728ddf7,e6211e25cf8fd7bc,E6853,content://media/external/audio/media/1600,<unknown>,GINTAMA 曇天,bluetooth,Walkman,2016-10-16 12:31:56.512,187
1,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,2016-10-16 12:31:56.512,1205,92e17ec9-982c-4e02-8ce4-76b19728ddf7,e6211e25cf8fd7bc,E6853,content://media/external/audio/media/1601,<unknown>,GunXSword,bluetooth,Walkman,2016-10-16 12:35:03.980,99
2,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,2016-10-16 12:31:56.512,1205,92e17ec9-982c-4e02-8ce4-76b19728ddf7,e6211e25cf8fd7bc,E6853,content://media/external/audio/media/1603,<unknown>,i jubeat knit~LOVE&JOY-木村由姬,bluetooth,Walkman,2016-10-16 12:36:42.627,35


## Investigate Track Artist

In [68]:
start_time = time.time()

n_rows_unknown_artist = (
    df_clean
    .where(
        (f.col('track_artist').isNull())
        | (f.col('track_artist') == '')
        | (f.col('track_artist') == '<unknown>')
    )
    .count()
)

print(f'Number rows where artist is unknown: {n_rows_unknown_artist}.')
print(f'That constitutes '
      f'{n_rows_unknown_artist / number_of_rows_full * 100:.5f} % '
      f'of the total number of cells.')
print(f'Execution time: {time.time() - start_time:.5f} s.')

Number rows where artist is unknown: 801504143.
That constitutes 22.44306 % of the total number of cells.
Execution time: 1031.07266 s.
